In [56]:
import pandas as pd
import numpy as np
import string
import scipy

import re
from pprint import pprint

import nltk
from nltk.stem import PorterStemmer

import gensim, logging
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, ldamodel

#import spacy (doesn't work...)

import pyLDAvis
#import pyLDA.gensim (doesn't work...)
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [17]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 

In [28]:
bailey = open('bailey.txt','r')
#print(bailey.read())

bailey_string = bailey.read()
btokens = nltk.word_tokenize(bailey_string)
    
print(btokens)

['Betty', 'Lou', 'Bailey', '(', '1929', '–', '2007', ')', '[', '1', ']', 'was', 'a', 'General', 'Electric', 'Company', 'mechanical', 'engineer', 'from', 'the', 'United', 'States', '.', 'She', 'held', 'a', 'patent', 'for', 'the', 'converging', 'diverging', 'variable', 'exhaust', 'nozzle', '.', 'The', 'invention', 'operated', 'so', 'that', 'one', 'would', 'vary', 'both', 'the', 'throat', 'and', 'the', 'exit', 'diameters', 'for', 'the', 'hot', 'gas', 'flows', '.', '[', '2', ']', 'In', 'honor', 'of', 'her', 'legacy', ',', 'the', 'Society', 'of', 'Women', 'Engineers', 'named', 'a', 'scholarship', 'after', 'her', '.', 'To', 'date', ',', 'that', 'scholarship', 'is', 'still', 'being', 'distributed', 'to', 'eligible', 'female', 'graduate', 'students', 'who', 'pursue', 'a', 'career', 'in', 'engineering', '.', '[', '3', ']', 'Bailey', 'was', 'the', 'first', 'female', 'member', 'of', 'the', 'Engineering', 'Society', 'of', 'Cincinnati', '.', 'She', 'later', 'served', 'as', 'the', 'chair', 'of', 'it

In [27]:
# This isn't working; it turns my words into letters. Deal with later.

#bigram = gensim.models.Phrases(btokens, min_count=5, threshold=100)
#trigram = gensim.models.Phrases(bigram[btokens], threshold=100)
#print(trigram[bigram[btokens[0]]])

#-----------One suggestion:

#from gensim.models import word2vec, Phrases
# See link (need to split into sentences but not sure how with double quotes...)
# https://stackoverflow.com/questions/46441876/why-do-i-get-single-letter-vocabulary-in-gensim-word2vec?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa

#bigram = gensim.models.Phrases(btokens, min_count=5, threshold=100)
#trigram = gensim.models.Phrases(bigram[btokens], threshold=100)

#for btoken in btokens:
 #   bigrams_ = bigram[sent]
  #  trigrams_ = trigrams[bigram[sent]]

#print(bigrams_)
#print(trigrams_)

#There's more code on that site...



/Users/marieskoczylas/anaconda3/envs/nlp_environ/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


NameError: name 'sent' is not defined

In [22]:
# Same issue as above (turns words to letters)

#bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)
#print(trigram_mod[bigram_mod[btokens[0]]])

In [25]:
stop_words = set(stopwords.words('english'))
 
# Now how to remove them? For loop?

['S', 'o', 'c', 'i', 'e', 't', 'y']


In [29]:
stop_words = set(stopwords.words('english'))
 
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#I'm supposed to make bigrams and trigrams here too, and lemmatize...

In [31]:
btokens_nostops = remove_stopwords(btokens)
print(btokens_nostops)
#This looks good but I need to get rid of the empty spaces that just return []

[['betty'], ['lou'], ['bailey'], [], [], [], [], [], [], [], [], [], [], ['general'], ['electric'], ['company'], ['mechanical'], ['engineer'], [], [], ['united'], ['states'], [], [], ['held'], [], ['patent'], [], [], ['converging'], ['diverging'], ['variable'], ['exhaust'], ['nozzle'], [], [], ['invention'], ['operated'], [], [], ['one'], ['would'], ['vary'], [], [], ['throat'], [], [], ['exit'], ['diameters'], [], [], ['hot'], ['gas'], ['flows'], [], [], [], [], [], ['honor'], [], [], ['legacy'], [], [], ['society'], [], ['women'], ['engineers'], ['named'], [], ['scholarship'], [], [], [], [], ['date'], [], [], ['scholarship'], [], ['still'], [], ['distributed'], [], ['eligible'], ['female'], ['graduate'], ['students'], [], ['pursue'], [], ['career'], [], ['engineering'], [], [], [], [], ['bailey'], [], [], ['first'], ['female'], ['member'], [], [], ['engineering'], ['society'], [], ['cincinnati'], [], [], ['later'], ['served'], [], [], ['chair'], [], [], ['guidance'], ['committee'], 

In [33]:
# Creating the dictionary
id2word = corpora.Dictionary(btokens_nostops)

# Creating the corpus
texts = btokens_nostops

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1)]]


In [37]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus]
# There's something wrong with my frequency count - it says "1" for every word

[[('betty', 1)],
 [('lou', 1)],
 [('bailey', 1)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('general', 1)],
 [('electric', 1)],
 [('company', 1)],
 [('mechanical', 1)],
 [('engineer', 1)],
 [],
 [],
 [('united', 1)],
 [('states', 1)],
 [],
 [],
 [('held', 1)],
 [],
 [('patent', 1)],
 [],
 [],
 [('converging', 1)],
 [('diverging', 1)],
 [('variable', 1)],
 [('exhaust', 1)],
 [('nozzle', 1)],
 [],
 [],
 [('invention', 1)],
 [('operated', 1)],
 [],
 [],
 [('one', 1)],
 [('would', 1)],
 [('vary', 1)],
 [],
 [],
 [('throat', 1)],
 [],
 [],
 [('exit', 1)],
 [('diameters', 1)],
 [],
 [],
 [('hot', 1)],
 [('gas', 1)],
 [('flows', 1)],
 [],
 [],
 [],
 [],
 [],
 [('honor', 1)],
 [],
 [],
 [('legacy', 1)],
 [],
 [],
 [('society', 1)],
 [],
 [('women', 1)],
 [('engineers', 1)],
 [('named', 1)],
 [],
 [('scholarship', 1)],
 [],
 [],
 [],
 [],
 [('date', 1)],
 [],
 [],
 [('scholarship', 1)],
 [],
 [('still', 1)],
 [],
 [('distributed', 1)],
 [],
 [('eligible', 1)],
 [('female', 1)],
 [('g

In [59]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [60]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.143*"tests" + 0.061*"exhaust" + 0.028*"familiar" + 0.028*"started" + '
  '0.028*"building" + 0.028*"enough" + 0.028*"compartment" + 0.028*"recognize" '
  '+ 0.028*"air" + 0.028*"piping"'),
 (1,
  '0.164*"company" + 0.119*"work" + 0.058*"general" + 0.035*"career" + '
  '0.020*"helped" + 0.020*"highest" + 0.020*"cycling" + 0.020*"november" + '
  '0.020*"downstream" + 0.020*"ensure"'),
 (2,
  '0.070*"guidance" + 0.061*"pass" + 0.061*"fellows" + 0.061*"philadelphia" + '
  '0.061*"york" + 0.059*"school" + 0.041*"one" + 0.037*"patented" + '
  '0.027*"science" + 0.021*"paul"'),
 (3,
  '0.286*"council" + 0.065*"attended" + 0.050*"jet" + 0.026*"progressed" + '
  '0.026*"initial" + 0.026*"turbines" + 0.026*"interview" + 0.026*"engine" + '
  '0.026*"job" + 0.015*"scholarship"'),
 (4,
  '0.051*"water" + 0.051*"adk" + 0.026*"commission" + 0.026*"licensing" + '
  '0.026*"trip" + 0.026*"suddenly" + 0.026*"recreation" + 0.026*"support" + '
  '0.026*"sufficient" + 0.026*"dam"'),
 (5,
  '0.475